In [1]:
# Install required libraries (run this in Colab)
!pip install transformers datasets scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
# Import necessary libraries
import pandas as pd
from transformers import pipeline
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [7]:
# Load sentiment analysis model (Hugging Face BERT)
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment', truncation=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Load the dataset (update with the correct file path in your Colab environment)
df = pd.read_csv('sentiment_385_sampled_cleanup.csv')

In [8]:
# Function to apply the BERT model with error handling
def classify_text(text, index):
    try:
        # BERT can process up to 512 tokens, set truncation to True to handle longer texts
        result = classifier(text)[0]['label']
        return result
    except Exception as e:
        print(f"Error at index {index}: {e}")
        print(f"Problematic text: {text}")
        return None  # Return None if there's an error

# Apply the BERT model to the comment_text column with error checking
df['predicted_sentiment'] = [classify_text(text, idx) for idx, text in enumerate(df['comment_text'])]

# Drop rows where sentiment prediction failed (optional)
df = df.dropna(subset=['predicted_sentiment'])

# Map the predicted sentiment (1-5 stars) to ternary format (-1, 0, 1)
def map_bert_to_custom(sentiment_label):
    if sentiment_label in ['1 star', '2 stars']:
        return -1  # Negative
    elif sentiment_label == '3 stars':
        return 0  # Neutral
    elif sentiment_label in ['4 stars', '5 stars']:
        return 1  # Positive

df['predicted_sentiment_mapped'] = df['predicted_sentiment'].apply(map_bert_to_custom)

# Ensure consensus_agreement is ternary (1 = positive, 0 = neutral, -1 = negative)
df['consensus_agreement_ternary'] = df['consensus_agreement'].apply(lambda x: int(x))

# Filter out rows where the model can't predict neutral (optional, if you want to exclude neutrals)
filtered_df = df[df['consensus_agreement_ternary'] != 0]

# Calculate evaluation metrics only for rows that have positive or negative sentiment
y_true = filtered_df['consensus_agreement_ternary']
y_pred = filtered_df['predicted_sentiment_mapped']

# F1 Score
f1 = f1_score(y_true, y_pred, average='weighted')  # weighted to handle class imbalance
# Precision
precision = precision_score(y_true, y_pred, average='weighted')
# Recall
recall = recall_score(y_true, y_pred, average='weighted')
# Accuracy
accuracy = accuracy_score(y_true, y_pred)

# Display the results
print(f"F1 Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Accuracy: {accuracy:.2f}")

# Export the DataFrame to a new CSV with predicted sentiment included
output_csv = 'sentiment_analysis_with_predictions.csv'
df.to_csv(output_csv, index=False)

print(f"Results saved to {output_csv}")

F1 Score: 0.74
Precision: 0.81
Recall: 0.68
Accuracy: 0.68
Results saved to sentiment_analysis_with_predictions.csv


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
